In [1]:
import torch
import torch.nn as nn
import torchvision

import torchbearer

from playground_utils import * # get common imports / vars for all nbs
all_module_versions(globals())

torch (1.3.0)
torchvision (0.4.1)
torchbearer (0.5.0)


In [8]:
save_path = Path('~/Documents/data/torchvision/')

DEVICE = print_and_return( torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') )
NUM_WORKERS = 0 # print_and_return( os.cpu_count() )
BATCH_SIZE  = 256

cpu


In [6]:
normalize  = torchvision.transforms.Normalize(**IMAGENET_NORMALIZATION)
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                             normalize])

# DATASETS
ds = torchvision.datasets.CIFAR10(str(save_path), train=True,  download=True, transform=transforms)
splitter = torchbearer.cv_utils.DatasetValidationSplitter(len(ds), 0.1)

train_ds = splitter.get_train_dataset(ds)
valid_ds = splitter.get_val_dataset(ds)
test_ds  = torchvision.datasets.CIFAR10(str(save_path), train=False, download=True, transform=transforms)

# DATALOADERS
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dl  = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

classes = ds.classes

print(len(train_ds))
print(len(valid_ds))
print(len(test_ds))

Files already downloaded and verified
Files already downloaded and verified
45000
5000
10000


In [7]:
get_dataset_distribution(train_ds, classes)

100%|██████████| 45000/45000 [00:06<00:00, 6549.64it/s]


Counter({'frog': 4468,
         'truck': 4510,
         'ship': 4497,
         'automobile': 4531,
         'bird': 4506,
         'airplane': 4537,
         'horse': 4504,
         'deer': 4498,
         'dog': 4494,
         'cat': 4455})

In [ ]:
dir(train_ds.dataset)